# Cleaning Player and Player Attributes Data

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

player_attr_df = pd.read_csv('../data/raw/Player_Attributes.csv')
player_df = pd.read_csv('../data/raw/Player.csv')

In [19]:
print("Player Attributes dataset:")
print(player_attr_df.shape)
player_attr_df.head()

Player Attributes dataset:
(183978, 42)


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [20]:
player_attr_df['date'] = pd.to_datetime(player_attr_df['date'])
player_attr_df['player_api_id'] = pd.to_numeric(player_attr_df['player_api_id'], errors='coerce').astype('Int64')
player_attr_df.head(3)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0


In [21]:
player_attr_df.tail(3)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
183975,183976,102359,39902,2008-08-30,77.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30,78.0,81.0,right,medium,low,74.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0
183977,183978,102359,39902,2007-02-22,80.0,81.0,right,medium,low,74.0,...,88.0,53.0,38.0,32.0,30.0,9.0,9.0,78.0,7.0,15.0


In [22]:
player_attr_df = player_attr_df.drop(['player_fifa_api_id'], axis=1)
player_attr_df = player_attr_df.drop(['id'], axis=1)
#player_attr_df = player_attr_df.set_index(['player_api_id', 'date']) #multi-index
player_attr_df

,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,2016-02-18,67.0,71.0,right,medium,medium,49.0,44.0,71.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,2015-11-19,67.0,71.0,right,medium,medium,49.0,44.0,71.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,2015-09-21,62.0,66.0,right,medium,medium,49.0,44.0,71.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,2015-03-20,61.0,65.0,right,medium,medium,48.0,43.0,70.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,2007-02-22,61.0,65.0,right,medium,medium,48.0,43.0,70.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,39902,2009-08-30,83.0,85.0,right,medium,low,84.0,77.0,59.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,39902,2009-02-22,78.0,80.0,right,medium,low,74.0,76.0,53.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,39902,2008-08-30,77.0,80.0,right,medium,low,74.0,71.0,53.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,39902,2007-08-30,78.0,81.0,right,medium,low,74.0,64.0,57.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [23]:
num_nulls = player_attr_df.isna().sum()
print(f"Number of NaNs in each column:\n{num_nulls}")

Number of NaNs in each column:
player_api_id             0
date                      0
overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    6869
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle         836
sliding_tackle   

In [24]:
player_attr_df.dropna(inplace=True)
player_attr_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177037 entries, 0 to 183977
Data columns (total 40 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   player_api_id        177037 non-null  Int64         
 1   date                 177037 non-null  datetime64[ns]
 2   overall_rating       177037 non-null  float64       
 3   potential            177037 non-null  float64       
 4   preferred_foot       177037 non-null  object        
 5   attacking_work_rate  177037 non-null  object        
 6   defensive_work_rate  177037 non-null  object        
 7   crossing             177037 non-null  float64       
 8   finishing            177037 non-null  float64       
 9   heading_accuracy     177037 non-null  float64       
 10  short_passing        177037 non-null  float64       
 11  volleys              177037 non-null  float64       
 12  dribbling            177037 non-null  float64       
 13  curve              

In [25]:
#organize data in numerical order by api id and then chronologically by date
player_attr_df = player_attr_df.sort_index(level=['player_api_id', 'date'])
player_attr_df

,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,2016-02-18,67.0,71.0,right,medium,medium,49.0,44.0,71.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,2015-11-19,67.0,71.0,right,medium,medium,49.0,44.0,71.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,2015-09-21,62.0,66.0,right,medium,medium,49.0,44.0,71.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,2015-03-20,61.0,65.0,right,medium,medium,48.0,43.0,70.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,2007-02-22,61.0,65.0,right,medium,medium,48.0,43.0,70.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,39902,2009-08-30,83.0,85.0,right,medium,low,84.0,77.0,59.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,39902,2009-02-22,78.0,80.0,right,medium,low,74.0,76.0,53.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,39902,2008-08-30,77.0,80.0,right,medium,low,74.0,71.0,53.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,39902,2007-08-30,78.0,81.0,right,medium,low,74.0,64.0,57.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


# Encoding categorical variables (3 total)

# preferred_foot
We mapped right to 1 and 0 to left for preferred_foot as a form of one-hot encoding.

In [26]:
player_attr_df['preferred_foot'] = player_attr_df['preferred_foot'].map({'right': 1, 'left': 0})
player_attr_df


,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,2016-02-18,67.0,71.0,1,medium,medium,49.0,44.0,71.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,2015-11-19,67.0,71.0,1,medium,medium,49.0,44.0,71.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,2015-09-21,62.0,66.0,1,medium,medium,49.0,44.0,71.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,2015-03-20,61.0,65.0,1,medium,medium,48.0,43.0,70.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,2007-02-22,61.0,65.0,1,medium,medium,48.0,43.0,70.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,39902,2009-08-30,83.0,85.0,1,medium,low,84.0,77.0,59.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,39902,2009-02-22,78.0,80.0,1,medium,low,74.0,76.0,53.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,39902,2008-08-30,77.0,80.0,1,medium,low,74.0,71.0,53.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,39902,2007-08-30,78.0,81.0,1,medium,low,74.0,64.0,57.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


## attacking_work_rate
We imputed the nonsense descriptions in 'attacking_work_rate' that were not "high," "medium," or "low" with "medium" because "medium" was the most frequent value, constituting more than two thirds of the rows. 

In [27]:
unique_values = player_attr_df['attacking_work_rate'].unique()
#print(f"Unique values in 'attacking_work_rate': {unique_values}")
value_counts = player_attr_df['attacking_work_rate'].value_counts()
player_attr_df['attacking_work_rate'] = player_attr_df['attacking_work_rate'].astype(str)
# Replace any value not 'high', 'medium', or 'low' with 'medium' (as imputed previously)
player_attr_df['attacking_work_rate'] = player_attr_df['attacking_work_rate'].apply(
    lambda x: x if x in ['high', 'medium', 'low'] else 'medium'
)
# Now map to integers
player_attr_df['attacking_work_rate'] = player_attr_df['attacking_work_rate'].map({'high': 2, 'medium': 1, 'low': 0}).astype(int)


In [28]:
unique_values = player_attr_df['defensive_work_rate'].unique()
player_attr_df['defensive_work_rate'] = player_attr_df['defensive_work_rate'].astype(str)
print(player_attr_df['defensive_work_rate'].value_counts())
player_attr_df['defensive_work_rate'] = player_attr_df['defensive_work_rate'].apply(
    lambda x: x if x in ['high', 'medium', 'low'] else 'medium'
)
# Now map to integers
player_attr_df['defensive_work_rate'] = player_attr_df['defensive_work_rate'].map({'high': 2, 'medium': 1, 'low': 0}).astype(int)
player_attr_df
print(player_attr_df['defensive_work_rate'].value_counts())
player_attr_df

defensive_work_rate
medium    130764
high       26972
low        18425
ormal        317
es            94
2             89
tocky         86
ean           78
1             44
6             34
5             28
4             27
3             23
0             20
9             17
7             14
8              5
Name: count, dtype: int64
defensive_work_rate
1    131640
2     26972
0     18425
Name: count, dtype: int64


,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,2016-02-18,67.0,71.0,1,1,1,49.0,44.0,71.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,2015-11-19,67.0,71.0,1,1,1,49.0,44.0,71.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,2015-09-21,62.0,66.0,1,1,1,49.0,44.0,71.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,2015-03-20,61.0,65.0,1,1,1,48.0,43.0,70.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,2007-02-22,61.0,65.0,1,1,1,48.0,43.0,70.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,39902,2009-08-30,83.0,85.0,1,1,0,84.0,77.0,59.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,39902,2009-02-22,78.0,80.0,1,1,0,74.0,76.0,53.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,39902,2008-08-30,77.0,80.0,1,1,0,74.0,71.0,53.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,39902,2007-08-30,78.0,81.0,1,1,0,74.0,64.0,57.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [29]:
print("Player dataset:")
print(player_df.shape)
player_df.drop(['id'], axis=1, inplace=True)
player_df.head()

Player dataset:
(11060, 7)


,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [30]:
player_df['birthday'] = pd.to_datetime(player_df['birthday'])
player_df['player_name'] = player_df['player_name'].astype(str)
player_df.sort_values(['player_api_id'], inplace=True)
player_df.drop(['player_fifa_api_id'], axis=1, inplace=True)
player_df

,player_api_id,player_name,birthday,height,weight
8359,2625,"Patryk Rachwal,18",1981-01-27,175.26,154
2636,2752,Diego Mainz,1982-12-29,187.96,174
5312,2768,Jose Dorado,1982-07-10,180.34,154
4335,2770,Ignacio Gonzalez,1982-05-14,180.34,161
305,2790,Alberto Rey,1974-02-15,175.26,163
...,...,...,...,...,...
3978,744907,Greg Taylor,1997-11-05,172.72,137
3484,746419,Francisco Afonso,1997-04-24,175.26,152
3603,748432,Fynn Arkenberg,1996-03-04,187.96,176
8827,750435,Rees Greenwood,1996-01-20,172.72,150


In [31]:
# Merge player_attr_df with player_df to add player_name, birthday, height, and weight
player_attr_df = player_attr_df.reset_index()
player_attr_df = player_attr_df.merge(
    player_df[['player_api_id', 'player_name', 'birthday', 'height', 'weight']],
    on='player_api_id',
    how='right'
)

In [32]:
player_attr_df.dropna(inplace=True)
player_attr_df.drop(['index'], axis=1, inplace=True)
player_attr_df
cols = player_attr_df.columns.tolist()

# Move the last 4 columns to the front
new_order = cols[-4:] + cols[:-4]
player_attr_df = player_attr_df[new_order]
player_attr_df
# Move the 5th column ('player_api_id') to the leftmost position
cols = player_attr_df.columns.tolist()
cols = [cols[4]] + cols[:4] + cols[5:]
player_attr_df = player_attr_df[cols]
player_attr_df

,player_api_id,player_name,birthday,height,weight,date,overall_rating,potential,preferred_foot,attacking_work_rate,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,2625,"Patryk Rachwal,18",1981-01-27,175.26,154,2015-01-16,61.0,61.0,1.0,1.0,...,55.0,66.0,62.0,63.0,54.0,12.0,11.0,6.0,8.0,8.0
1,2625,"Patryk Rachwal,18",1981-01-27,175.26,154,2014-11-07,61.0,61.0,1.0,1.0,...,55.0,66.0,62.0,63.0,54.0,12.0,11.0,6.0,8.0,8.0
2,2625,"Patryk Rachwal,18",1981-01-27,175.26,154,2014-09-18,61.0,61.0,1.0,1.0,...,55.0,66.0,62.0,63.0,54.0,12.0,11.0,6.0,8.0,8.0
3,2625,"Patryk Rachwal,18",1981-01-27,175.26,154,2013-06-07,61.0,61.0,1.0,1.0,...,55.0,66.0,62.0,63.0,54.0,12.0,11.0,6.0,8.0,8.0
4,2625,"Patryk Rachwal,18",1981-01-27,175.26,154,2013-03-08,61.0,61.0,1.0,1.0,...,55.0,66.0,62.0,63.0,54.0,12.0,11.0,6.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178044,750435,Rees Greenwood,1996-01-20,172.72,150,2015-02-06,56.0,70.0,1.0,1.0,...,58.0,61.0,18.0,19.0,21.0,9.0,10.0,8.0,10.0,11.0
178045,750435,Rees Greenwood,1996-01-20,172.72,150,2015-01-16,56.0,70.0,1.0,1.0,...,58.0,61.0,18.0,19.0,21.0,9.0,10.0,8.0,10.0,11.0
178046,750435,Rees Greenwood,1996-01-20,172.72,150,2007-02-22,56.0,70.0,1.0,1.0,...,58.0,61.0,18.0,19.0,21.0,9.0,10.0,8.0,10.0,11.0
178047,750584,Alexandre Azevedo,1997-01-28,175.26,150,2016-05-19,58.0,66.0,1.0,1.0,...,47.0,52.0,56.0,66.0,62.0,8.0,9.0,13.0,11.0,13.0


In [33]:
#Add player age column that stores age of player in years for each rating
player_attr_df['age'] = (
    pd.to_datetime(player_attr_df['date']) - pd.to_datetime(player_attr_df['birthday'])
).dt.days / 365.25
# Move 'age' column to the right of 'date'
cols = list(player_attr_df.columns)
cols.insert(cols.index('date') + 1, cols.pop(cols.index('age')))
player_attr_df = player_attr_df[cols]
player_attr_df = player_attr_df.drop(columns=['player_name', 'birthday'])

In [34]:
#ensuring that all numeric are float64
player_attr_df['weight'] = player_attr_df['weight'].astype('float64')

In [35]:
player_attr_df.to_csv('../data/cleaned/player_attributes_cleaned.csv', index=False)
player_attr_df

,player_api_id,height,weight,date,age,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,2625,175.26,154.0,2015-01-16,33.968515,61.0,61.0,1.0,1.0,1.0,...,55.0,66.0,62.0,63.0,54.0,12.0,11.0,6.0,8.0,8.0
1,2625,175.26,154.0,2014-11-07,33.776865,61.0,61.0,1.0,1.0,1.0,...,55.0,66.0,62.0,63.0,54.0,12.0,11.0,6.0,8.0,8.0
2,2625,175.26,154.0,2014-09-18,33.639973,61.0,61.0,1.0,1.0,1.0,...,55.0,66.0,62.0,63.0,54.0,12.0,11.0,6.0,8.0,8.0
3,2625,175.26,154.0,2013-06-07,32.358658,61.0,61.0,1.0,1.0,1.0,...,55.0,66.0,62.0,63.0,54.0,12.0,11.0,6.0,8.0,8.0
4,2625,175.26,154.0,2013-03-08,32.109514,61.0,61.0,1.0,1.0,1.0,...,55.0,66.0,62.0,63.0,54.0,12.0,11.0,6.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178044,750435,172.72,150.0,2015-02-06,19.047228,56.0,70.0,1.0,1.0,0.0,...,58.0,61.0,18.0,19.0,21.0,9.0,10.0,8.0,10.0,11.0
178045,750435,172.72,150.0,2015-01-16,18.989733,56.0,70.0,1.0,1.0,0.0,...,58.0,61.0,18.0,19.0,21.0,9.0,10.0,8.0,10.0,11.0
178046,750435,172.72,150.0,2007-02-22,11.091034,56.0,70.0,1.0,1.0,0.0,...,58.0,61.0,18.0,19.0,21.0,9.0,10.0,8.0,10.0,11.0
178047,750584,175.26,150.0,2016-05-19,19.304586,58.0,66.0,1.0,1.0,1.0,...,47.0,52.0,56.0,66.0,62.0,8.0,9.0,13.0,11.0,13.0
